In [1]:
# 필수 라이브러리 임포트
from typing import Tuple
import matplotlib
import meep as mp
import nlopt
import numpy as np
import os

# matplotlib의 백엔드를 "agg"로 설정하여 이미지를 파일로 저장 가능하게 합니다.
matplotlib.use("agg")
import matplotlib.pyplot as plt
 
# Lorentzian 함수 정의: Lorentzian 파라미터를 사용하여 복소 유전율 프로파일을 계산
def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    N = len(p) // 3  # 파라미터가 3개씩 묶인 Lorentzian 항의 개수
    y = np.zeros(len(x))  # 결과값을 저장할 배열
    for n in range(N):
        A_n = p[3 * n + 0]  # 첫 번째 파라미터 (진폭)
        x_n = p[3 * n + 1]  # 두 번째 파라미터 (중심 주파수)
        g_n = p[3 * n + 2]  # 세 번째 파라미터 (감쇠율)
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
    return y

# 잔차 함수 정의: 실제 값과 Lorentzian 모델 간의 차이를 계산하고 그라디언트도 반환
def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    N = len(p) // 3  # Lorentzian 항의 개수
    yp = lorentzfunc(p, x)  # 예측된 유전율 프로파일 계산
    val = np.sum(np.square(abs(y - yp)))  # 실제 값과 예측 값의 차이(L2 norm)

    # 그라디언트 계산
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))  # 진폭에 대한 그라디언트
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))  # 중심 주파수에 대한 그라디언트
            grad[3 * n + 2] = -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))  # 감쇠율에 대한 그라디언트
    return val  # 최종 오차 반환

# Lorentzian 파라미터 최적화 함수 정의
def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    # NLopt 최적화 설정
    opt = nlopt.opt(alg, len(p0))  # 최적화 알고리즘 설정 (LD_LBFGS는 LBFGS 알고리즘)
    opt.set_ftol_rel(tol)  # 상대 오차 허용 범위 설정
    opt.set_maxeval(maxeval)  # 최대 반복 횟수 설정
    opt.set_lower_bounds(np.zeros(len(p0)))  # 하한값 설정
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))  # 상한값 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))  # 목표 함수 설정
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))  # 로컬 최적화 설정
    local_opt.set_ftol_rel(1e-10)  # 로컬 최적화 상대 오차 설정
    local_opt.set_xtol_rel(1e-8)  # 로컬 최적화 X 변화 상대 오차 설정
    opt.set_local_optimizer(local_opt)  # 로컬 최적화 설정 추가
    popt = opt.optimize(p0)  # 최적화 실행
    minf = opt.last_optimum_value()  # 마지막 최적화 값 반환
    return popt, minf  # 최적화된 파라미터와 최소 오차 반환

In [2]:
def Material_fit(Material_data_csv="SiO2_palik_data.csv", 
                   FDTD_material_csv="fdtd_data_SiO2_Palik.csv",
                   eps_inf=1,
                   fit_wl_min=0.38, fit_wl_max=0.73,
                   num_lorentzians=2, iteration=50,
                   save_path="."):    
    # 저장 경로가 없으면 생성
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 1. 데이터 로드 및 전처리
    data = np.genfromtxt(Material_data_csv, delimiter=",")
    wl = data[:, 0] * 1e6  # 파장 (µm)
    n_data = data[:, 1] + 1j * data[:, 2]
    eps_all = np.square(n_data) - eps_inf

    # 피팅 범위로 데이터 선택
    mask = (wl >= fit_wl_min) & (wl <= fit_wl_max)
    wl_fit, eps_fit = wl[mask], eps_all[mask]
    freqs_fit = 1 / wl_fit

    # 2. Lorentzian 피팅 최적화
    ps = np.zeros((iteration, 3 * num_lorentzians))
    errors = np.zeros(iteration)
    for m in range(iteration):
        p_rand = [10 ** np.random.random() for _ in range(3 * num_lorentzians)]
        ps[m, :], errors[m] = lorentzfit(p_rand, freqs_fit, eps_fit, nlopt.LD_MMA, 1e-25, 50000)
        print(f"Iteration {m:3d}, error: {errors[m]:.6f}")
    best = np.argmin(errors)
    print(f"Optimal error: {errors[best]:.6f}")

    # 3. 최적 파라미터로 Susceptibility 생성 및 Meep 모델 구성
    suscept = []
    for i in range(num_lorentzians):
        freq_param = ps[best][3*i + 1]
        gamma = ps[best][3*i + 2]
        if freq_param == 0:
            sigma = ps[best][3*i + 0]
            suscept.append(mp.DrudeSusceptibility(frequency=1.0, gamma=gamma, sigma=sigma))
        else:
            sigma = ps[best][3*i + 0] / freq_param**2
            suscept.append(mp.LorentzianSusceptibility(frequency=freq_param, gamma=gamma, sigma=sigma))
    material = mp.Medium(epsilon=eps_inf, E_susceptibilities=suscept)
    model_eps = [material.epsilon(f)[0][0] for f in freqs_fit]

    # 4. Material 데이터와 Meep 모델 비교 플롯 생성
    plt.close('all')
    fig, ax = plt.subplots(1, 2, figsize=(8, 3))
    
    # Re(ε) 플롯
    ax[0].plot(wl_fit, np.real(eps_fit) + eps_inf, 'gs', markersize=4, label="Material data")
    ax[0].plot(wl_fit, np.real(model_eps), 'b-', label="Meep model")
    ax[0].set_xlabel("wavelength (µm)")
    ax[0].set_ylabel(r"Re($\epsilon$)")
    ax[0].grid(True)
    ax[0].set_xlim([fit_wl_min+0.02, fit_wl_max-0.03])
    
    # Im(ε) 플롯
    ax[1].plot(wl_fit, np.imag(eps_fit), 'gs', markersize=4, label="Material data")
    ax[1].plot(wl_fit, np.imag(model_eps), 'b-', label="Meep model")
    ax[1].set_xlabel("wavelength (µm)")
    ax[1].set_ylabel(r"Im($\epsilon$)")
    ax[1].grid(True)
    ax[1].set_xlim([fit_wl_min+0.02, fit_wl_max-0.03])
    
    fig.suptitle("Comparison of Material Data and FDTD Model\n(using Drude-Lorentzian Susceptibility)", fontsize=9)
    fig.subplots_adjust(wspace=0.3)
    fig.savefig(os.path.join(save_path, "eps_fit_sample.png"), dpi=150, bbox_inches="tight")

    # 5. FDTD 비교 데이터 로드 및 플롯 추가
    csv = np.genfromtxt(FDTD_material_csv, delimiter=",", skip_header=1)
    csv_wl = csv[:, 0]
    csv_n = csv[:, 1] + 1j * csv[:, 2]
    csv_eps = np.square(csv_n) - eps_inf

    ax[0].plot(csv_wl, np.real(csv_eps) + eps_inf, 'r--', markersize=6, label="FDTD model")
    ax[1].plot(csv_wl, np.imag(csv_eps), 'r--', markersize=6, label="FDTD model")
    ax[0].legend()
    ax[1].legend()
    fig.savefig(os.path.join(save_path, "eps_fit_sample_with_csv.png"), dpi=150, bbox_inches="tight")
    print(f"FDTD 데이터가 추가된 플롯을 지정된 경로로 저장했습니다.")

    # 6. 결과 CSV 파일 저장
    fdtd_save = os.path.join(save_path, "fdtd_fit_data.csv")
    np.savetxt(fdtd_save, np.column_stack((csv_wl, np.real(csv_eps) + eps_inf, np.imag(csv_eps))), delimiter=",")
    
    save_data = np.column_stack((wl_fit, np.real(eps_fit) + eps_inf, np.imag(eps_fit),
                                  np.real(model_eps), np.imag(model_eps)))
    eps_save = os.path.join(save_path, "eps_fit_sample_data.csv")
    np.savetxt(eps_save, save_data, delimiter=",",
               header="wavelength(um),Material_Re,Material_Im,Meep_Re,Meep_Im", comments="")
    print(f"계산된 meep material fitting 정보 CSV 파일 저장 완료.")


Material_fit(save_path="/home/min/EIDL/Tool/Meep/LGD/Material CSV/Al_Palik/Data/CSV file/SiO2")


Iteration   0, error: 0.000012
Iteration   1, error: 0.000006
Iteration   2, error: 0.000010
Iteration   3, error: 0.000022
Iteration   4, error: 0.000026
Iteration   5, error: 0.000334
Iteration   6, error: 0.000018
Iteration   7, error: 0.000016
Iteration   8, error: 0.000013
Iteration   9, error: 0.000008
Iteration  10, error: 0.000007
Iteration  11, error: 0.000053
Iteration  12, error: 0.000020
Iteration  13, error: 0.000011
Iteration  14, error: 0.000025
Iteration  15, error: 0.000008
Iteration  16, error: 0.000010
Iteration  17, error: 0.000006
Iteration  18, error: 0.000004
Iteration  19, error: 0.000090
Iteration  20, error: 0.000008
Iteration  21, error: 0.000110
Iteration  22, error: 0.000006
Iteration  23, error: 0.000013
Iteration  24, error: 0.000027
Iteration  25, error: 0.000019
Iteration  26, error: 0.000020
Iteration  27, error: 0.000016
Iteration  28, error: 0.000016
Iteration  29, error: 0.000011
Iteration  30, error: 0.000016
Iteration  31, error: 0.000006
Iteratio